In [1]:
import numpy as np
import scipy.stats as spy
import scipy.io as sio

def num2signal(x):
    dim = np.max(x,0)[0]
    n = x.shape[0]
    s = np.zeros((n, dim))
    
    for i in range(n):
        s[i, x[i]-1] = 1
    return s

In [2]:
name = ['amazon_SURF_L10.mat', 'Caltech10_SURF_L10.mat', 'dslr_SURF_L10.mat', 'webcam_SURF_L10.mat']
result_name = ['amazon_SURF_L10', 'Caltech10_SURF_L10', 'dslr_SURF_L10', 'webcam_SURF_L10']

for i in range(4):
    file_name = name[i]
    result_file = result_name[i]
    
    mat = sio.loadmat(file_name)
    labels = mat['labels']
    labels = num2signal(labels)
    
    fts = mat['fts']
    
    #data preprocessing 
    fts = np.float32(fts) / np.tile(np.sum(fts, axis=1, dtype='float32', keepdims=True), (1, fts.shape[1]))
    fts = spy.zscore(fts, 0)    
    
    r_train = 0.5
    r_validate = 0.3
    r_test = 0.2
    
    n = np.shape(fts)[0]
    train_n = np.floor(n * r_train).astype(int)
    validataion_n = np.floor(n * r_validate).astype(int)    
    test_n = n - train_n - validataion_n
    
    validataion_l_n = validataion_n // 10
    test_l_n = test_n // 10

    train_fts = []
    train_labels = []
    validataion_fts = None
    validataion_labels = None
    test_fts = None
    test_labels = None
    
    #make the validation and test data balance(data number of any label are same)
    k=9
    ind = np.where(labels[:,k] == 1)[0][0:test_l_n]
        
    test_fts = fts[ ind , :]
    test_labels = labels[ ind , :]
    fts = np.delete(fts, (ind), axis = 0)
    labels = np.delete(labels, (ind), axis = 0)
        
    ind = np.where(labels[:,k] == 1)[0][0:validataion_l_n]
    validataion_fts = fts[ ind , :]
    validataion_labels =  labels[ ind , :]
    fts = np.delete(fts, (ind), axis = 0)
    labels = np.delete(labels, (ind), axis = 0)    
    
    for k in range(9):     
        ind = np.where(labels[:,k] == 1)[0][0:test_l_n]
        
        test_fts = np.append( test_fts, fts[ ind , :], axis=0 )
        test_labels = np.append( test_labels, labels[ ind , :], axis=0 )
        fts = np.delete(fts, (ind), axis = 0)
        labels = np.delete(labels, (ind), axis = 0)
        
        ind = np.where(labels[:,k] == 1)[0][0:validataion_l_n]
        validataion_fts = np.append( validataion_fts, fts[ ind , :], axis=0 )
        validataion_labels = np.append( validataion_labels, labels[ ind , :], axis=0 )
        fts = np.delete(fts, (ind), axis = 0)
        labels = np.delete(labels, (ind), axis = 0)  
        
    print(np.shape(test_fts))
    
    n = np.shape(fts)[0]
    train_n = np.floor(n * 0.3).astype(int)
    validataion_n = np.floor(n * 0.2).astype(int)    
    
    index = np.random.permutation(n)
    fts = fts[index, :]
    labels = labels[index, :]
        
    train_fts = fts
    train_labels = labels        

    pkl = [(train_fts, train_labels), (validataion_fts, validataion_labels), (test_fts, test_labels)]
    
    np.save(result_file, pkl)
    
print("Done")    

(190, 800)
(220, 800)
(30, 800)
(60, 800)
Done
